In [ ]:
#installing pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#start local spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [ ]:
##import sqlConteHight module
from pyspark.sql import SQLContext
from pyspark.sql.functions import *

sqlCtx = SQLContext(sc)


/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
#read the data into spark dataframe
stock_sdf = sqlCtx.read.csv('saf_stock.csv', header=True, inferSchema=True)



In [ ]:
#olumns name
print(stock_sdf.columns)
#the inferred schema
print(stock_sdf.printSchema())
#preview first five
stock_sdf.show(5)


['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']
root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)

None
+-------------------+------------------+---------+---------+------------------+--------+------------------+
|               Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00

In [ ]:
#describe the data
stock_sdf.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

In [ ]:
# #read the data into RDD object
# saf_stock_raw = sc.teHightFile('saf_stock.csv')
# saf_stock = saf_stock_raw.map(lambda line: line.split(','))
# saf_stock.take(5)


In [ ]:
# stock_sdf = stock_sdf.withColumn('High', round(stock_sdf.High, 2))
stock_sdf.show()

+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
|               Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000

In [ ]:
#data formatting 2 decimal place
# stock_sdf = stock_sdf.withColumn('adj_close', round(stock_sdf.adj_close, 2))

for c in stock_sdf.columns[1:]:
  stock_sdf = stock_sdf.withColumn(c, round(c, 2))
#add column HV Ratio that is the ratio of the High Price versus volume of stock traded for a day
mod_stock_sdf = stock_sdf.withColumn('HV Ratio', stock_sdf.High/stock_sdf.Volume)
mod_stock_sdf.show()

+-------------------+-----+-----+-----+-----+--------+---------+--------------------+
|               Date| Open| High|  Low|Close|  Volume|Adj Close|            HV Ratio|
+-------------------+-----+-----+-----+-----+--------+---------+--------------------+
|2012-01-03 00:00:00|59.97|61.06|59.87|60.33|12668800|    52.62|4.819714574387472E-6|
|2012-01-04 00:00:00|60.21|60.35|59.47|59.71| 9593300|    52.08|6.290848821573389...|
|2012-01-05 00:00:00|59.35|59.62|58.37|59.42|12768200|    51.83|4.669413073103491E-6|
|2012-01-06 00:00:00|59.42|59.45|58.87| 59.0| 8069400|    51.46|7.367338339901356E-6|
|2012-01-09 00:00:00|59.03|59.55|58.92|59.18| 6679300|    51.62|8.915604928660188E-6|
|2012-01-10 00:00:00|59.43|59.71|58.98|59.04| 6907300|    51.49|8.644477581688938E-6|
|2012-01-11 00:00:00|59.06|59.53|59.04| 59.4| 6365600|    51.81| 9.35182857861003E-6|
|2012-01-12 00:00:00|59.79| 60.0| 59.4| 59.5| 7236400|     51.9| 8.29141562102703E-6|
|2012-01-13 00:00:00|59.18|59.61|59.01|59.54| 7729300|

In [ ]:
#register the dataframe as a table
from pyspark.sql import SQLContext
sqlCtHigh = SQLContext(sc)
mod_stock_sdf.createOrReplaceTempView('mod_stock_table')
sqlCtx.tableNames()
 

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


['mod_stock_table']

In [ ]:
print(mod_stock_sdf.columns)

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close', 'HV Ratio']


In [ ]:
#What day had the Peak High in Price
query = """
select max(High), first(Date) as date
from mod_stock_table
"""
sqlCtx.sql(query).show()

+---------+-------------------+
|max(High)|               date|
+---------+-------------------+
|    90.97|2012-01-03 00:00:00|
+---------+-------------------+



In [ ]:
#mean of the Close column
query = """
select mean(Close)
from mod_stock_table
"""
sqlCtx.sql(query).show()

+-----------------+
|      mean(Close)|
+-----------------+
|72.38844992050863|
+-----------------+



In [ ]:
#max and min of the Volume column
query = """
select min(Volume), max(Volume)
from mod_stock_table
"""
sqlCtx.sql(query).show()

+-----------+-----------+
|min(Volume)|max(Volume)|
+-----------+-----------+
|    2094900|   80898100|
+-----------+-----------+



In [ ]:
##How many days was the Close lower than 60 dollars
query = """
select count(Date)
from mod_stock_table
where Close <60
"""
sqlCtx.sql(query).show()

+-----------+
|count(Date)|
+-----------+
|         81|
+-----------+



In [ ]:
#percentage of the time was the High greater than 80 dollars
total_days = mod_stock_sdf[['Date']].count()
high_days = mod_stock_sdf[mod_stock_sdf['High']>80].count()
perc_high = (high_days/total_days)*100
perc_high

9.141494435612083

In [ ]:
#What is the Pearson correlation between High and Volume
# from pyspark.ml.linalg import DenseMatrix, Vectors
# from pyspark.ml.stat import Correlation

# high_volume = mod_stock_sdf[['High', 'Volume']]
# high_volume.show()
# pearsonCorr = Correlation.corr(high_volume, 'Volume', 'pearson')
# pearsonCorr

query ="""
SELECT ((SUM(High * Volume)– (SUM(High) * SUM(Volume)) / COUNT(*)))/(SQRT(SUM(High * High)–(SUM(High) * SUM (High)) / COUNT(*))* SQRT(SUM(Volume * Volume)– (SUM(Volume) * SUM(Volume)) / COUNT(*))) AS pearson
FROM mod_stock_table
"""
sqlCtx.sql(query).show()

In [ ]:
#What is the max High per year
query = """
select YEAR(Date) as year, max(High) as high
from mod_stock_table
group by year
"""

sqlCtx.sql(query).show()

+----+-----+
|year| high|
+----+-----+
|2015|90.97|
|2013|81.37|
|2014|88.09|
|2012| 77.6|
|2016|75.19|
+----+-----+



In [ ]:
#What is the average Close for each Calendar Month
query = """
select mean(Close) as monthly_mean, MONTH(Date) as month
from mod_stock_table
group by month
order by month
"""

sqlCtx.sql(query).show()

+-----------------+-----+
|     monthly_mean|month|
+-----------------+-----+
|71.44801980198022|    1|
|71.30680412371134|    2|
|71.77794392523363|    3|
|72.97361904761907|    4|
|72.30971698113206|    5|
|72.49537735849057|    6|
|74.43971962616824|    7|
|73.02981818181819|    8|
|72.18411764705883|    9|
|71.57854545454546|   10|
|72.11108910891085|   11|
|72.84792452830189|   12|
+-----------------+-----+

